## Исследование надёжности заёмщиков.
Кредитный отдел банка заказал исследование  - необходимо понять влияет ли семейное положение, количество детей клиента и другие факторы на погашение кредита в срок.
Данные от банка — статистика о платёжеспособности.
Результаты исследования будут учтены при построении скоринговой модели.

# Шаг 1. Импортируем библиотеки

Откроем файл с данными и изучим общую информацию

In [1]:
# import pandas
# import pandas as pd
# import warnings
# warnings.filterwarnings('ignore') # отключу предупреджения Pandas
# data = pd.read_csv('/датафрейм.csv/') # Прочитал файл и сохранил его в переменной data
# data.info()
# data.head()

Посмотрим сводную информацию о данных

In [2]:
# data.describe()

### Вывод

Открыл таблицу и изучил общую информацию о данных - 12 столбцов, 21525 строк, с разными типами данных.
1. Столбец "столбец", обнаружил:
 - некоторые заемщики имеют "-1" ребенка, вероятно ошибка в данных, исправлю,
 - некоторые заемщики имеют 20 детей, вполне может быть, в дальнейшем категоризирую,
 - пропусков нет.
2. Столбец "столбец" (трудовой стаж), обнаружил:
 - больше 2174 пропуска, почти 10% строк, надо будет заполнить,
 - средний трудовой стаж, почти 173 года, очевидно в данных ошибка, буду искать, аналогично и для максимального стажа 1100 лет, многовато,
 - отрицательный трудовой стаж - неверные данные.
3. Столбец "столбец" (возраст клиента в годах), обнаружил:
- нулевое значение возраста, некоторых заемщиков, надо будет заполнить,
- максимальный возраст заемщика 75 лет, многовато, но, в принципе может быть.
4. Столбец "столбец" (уровень образования клиента), обнаружил, что:
- заполнено по разному - в разных регистрах, возможно с большим количеством вариантов, приведу к нижнему регистру и проведу лемматизацию,
- пропусков нет.
5. Столбец "столбец" - не несет смысловой нагрузки, удалю.
6. Столбец "столбец", обнаружил:
 - большую вариатовность указания семейного положения, буду лемматизировать, категоризировать,
 - пропусков нет.
7. Столбец "столбец" - не несет смысловой нагрузки, удалю.
8. Столбец "столбец" (пол), обнаружил:
- пропусков нет,
- проверю данные на предмет, только двух вариантов F и M, соответственно, если ОК, так и оставлю.
9. Столбец "столбец" (тип занятости), обнаружил:
 - много вариантов указания типа занятости, лемматезирую,
 - пропусков нет.
10. Столбец "столбец" (имел ли задолженность по возврату кредитов) - проверю на корректность еще раз,
- пропусков нет.
11. Столбец "столбец" (ежемесячный доход), обнаружил:
 - много пропусков, возможные причины:
  - возможно данные были получены из разных источников, например из НБКИ где нет данных о ежемесячном доходе, 
  - возможно джля клиентов предоставляющих "справки по форме банка", также не требовалось заполнение этого поля, 
  - возможно была реализована некая маркетиногвая компания с предодобренными кредитами, 
  - возможно это клиенты держатели зарплатных карт, которым, также был предоставлен кредит без предоставления дополнительной     информации о доходе,
  - возможно это ИП/самозанятые, которые не могут предоставить соответствующие справки.
 - в столбце вещественные числа, приведу к целочисленным,
 - категоризирую заемщиков.
12. Столбец "столбец" (цель получения кредита), обнаружил:
 - много вариантов указания цели получения кредита, буду лемматизировать, категоризировать,
 - пропусков нет.

### Шаг 2. Предобработка данных

Столбец "столбец"

In [3]:
# print("Количество людей с -1 ребенком:", датафрейм[датафрейм['столбец'] == -1].count()[0])
# print("Количество строк с 20 детьми:", датафрейм[датафрейм['столбец'] == 20].count()[0])
# print("Количество уникальных людей с 20 детьми:", len(датафрейм[датафрейм['столбец'] == 20]['total_income'].unique()))

Заменю людей с "-1" ребенком на "1", так как, вероятно, при внесении данных написали "тире 1", далее буду категоризировать, поэтому заменять значения для "20", пока не нужно

In [4]:
# [датафрейм['столбец'] = [датафрейм['столбец'].replace(-1, 1)

Столбец трудовой стаж

In [5]:
# print("Количество строк с 'столбец' > 0:", датафрейм[датафрейм['столбец'] > 0].shape[0])
# print("Количество строк с 'столбец' < 0:", датафрейм[датафрейм['столбец'] < 0].shape[0])
# print("Количество строк с 'столбец' = 0:", датафрейм[датафрейм['столбец'] == 0].shape[0])

У 15906 заемщиков обнаружился отрицательный трудовой стаж, чего не может быть, возьму трудовой стаж по модулю, над пропусками буду работать далее

In [6]:
# датафрейм['столбец'] = датафрейм['столбец'].abs()

Удалю столбцы неиспользуемые в анализе

In [7]:
# датафрейм.drop(['столбец, 'столбец'], axis='columns', inplace=True)

### Обработка пропусков

Пропуски и нулевые значения, обнаружились в столбцах "столбец", "столбец", "столбец"

Удаление пропусков в столбцах:  

Проверю строки, где отсутствуют данные по столбцам "столбец" и "столбец" на предмет совпадения

In [8]:
# датафрейм[(датафрейм['столбец'].isnull() == True) & (датафрейм['столбец'].isnull() == True)].info()

Таких строк нет, значит возраст возьму "средний", так как больше отталкиваться не от чего

In [9]:
# data_dob_years_mean = датафрейм['столбец'].mean()
# print(data_dob_years_mean)

Средний возраст заемщиков 43 года, что похоже на правду, произведу замену нулевых значений на средний возраст и посмотрю на данные по столбцу dob_years

In [10]:
# датафрейм['столбец'] = датафрейм['столбец'].replace(0, 43)
# датафрейм['столбец'].describe()

Удаление пропусков в столбце "столбец"

Проверю строки, где отсутствуют данные по столбцам "столбец" и "столбец" на предмет совпадения

In [11]:
# датафрейм['столбец'].isnull() == True) & (датафрейм['столбец'].isnull() == True)].info()

Так и есть - в строках, где отсутствуют данные по столбцу "столбец", отсутствуют данные и по столбцу "столбец"

Проверю из каких профессий эти заемщики

In [12]:
# датафрейм[(датафрейм['столбец'].isnull() == True) & (датафрейм['столбец'].isnull() == True)]['income_type'].value_counts()

Из разных, даже госслужащие есть

Сделаю группировку для каждого из типов "столбец", чтобы выяснить, откуда взялся средний стаж в 173 года  
Для каждого типа "столбец" выведу данные:  
- Общее кол-во строк данного типа в таблице,  
- Среднее значение по "столбец",  
- Для проверки корректности вычисления среднего выведу количество значений по "столбец" > 0 по данному "столбец"  

In [13]:
# data_grouped = датафрейм.groupby('столбец').agg({'столбец':['count', 'mean', lambda x: sum(x>0)]})

# dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['Общее количество строк', 'Среднее', 'Кол-во значений > 0']))

# data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
# data_grouped

Перекос в данных создают пенсионеры и безработные  
Добавлю столбец "столбец" для вычисления коэффициента полноты количества отработанных дней (учитывая то, что официально можно трудиться с 16 лет), чтобы впоследствии заполнить NaN по "столбец" с учетом среднего этого показателя по группе, избегая столбцов, где "столбец" NaN через notnull())  

In [14]:
# датафрейм['столбец'] = датафрейм[датафрейм['столбец'].notnull()]['столбец']/((датафрейм['столбец']-16)*365)
# data.head()

Заполню пропуски NaN в "столбец" - NaN заменю на произведение среднего коэффициента по группе "столбец", умноженного на возраст в днях  

In [15]:
# датафрейм['столбец'] = датафрейм.groupby('столбец')['столбец'].transform(lambda x: x.fillna(x.mean()*датафрейм['столбец']*365))
# data.head()

Удалю вспомогательный столбец, приведу столбец "столбец" к целочисленному виду и выведу общую информацию по датафрейму  

In [16]:
# data = датафрейм.drop('столбец', axis=1)
# датафрейм['столбец'] = датафрейм['столбец'].astype('int')
# data.info()

Посмотрю сводную информацию

In [17]:
# data.describe()

Удаление пропусков в столбце "столбец"

Большое количество пропусков обнаружилось в столбце "столбец".  
Посмотрю на гистограмму распределения значений дохода ('столбец'). Заметно, что выборка смещена вправо. Слишком широкий разброс.  
Посчитаю другими методами.  
В соответствии с: https://www.bnkomi.ru/data/news/59037/  
Граждане с уровнем дохода относяться к категориям:  
 - крайняя нищета — доходы ниже прожиточного минимума (до 7-8 тыс.р.)  
 - нищета — доходы от одного до двух прожиточных минимумов (от 8 до 12 тыс.р.)  
 - бедность — доходы от 12 до 20 тысяч рублей в месяц.  
 - выше бедности — доходы от 20 до 30 тысяч рублей в месяц.  
 - средний достаток — доходы от 30 до 60 тысяч рублей в месяц.  
 - состоятельные — доходы от 60 до 90 тысяч рублей в месяц.  
 - богатые — доходы от 90 тысяч рублей в месяц.  
 - сверхбогатые — доходы свыше 150 тысяч рублей в месяц.  
-------------------------------------------------------
Объединю в группы:  
 - бедность  - доходы от 0 до 30 тысяч рублей в месяц, включительно (для кода "poor"),  
 - средний достаток — доходы от 30 до 90 тысяч рублей в месяц, включительно (для кода "medium"),  
 - богатые -доходы от 90 до 150 тысяч рублей в месяц, включительно (для кода "rich"),  
 - сверхбогатые — доходы свыше 150 тысяч рублей в месяц.(для кода "moneybags")  
И посчитаю количество по группам.  

In [18]:
# датафрейм['столбец'].hist(density=True, bins=1000) # density со значением True нужен для построения гистограммы плотности вероятностей
# data_ti_poor = датафрейм['столбец'].loc[(датафрейм['столбец'] > 0) & (датафрейм['столбец'] <= 30000)]
# data_ti_poor_percent = (data_ti_poor.count() / 19351)
# data_ti_medium = датафрейм['столбец'].loc[(датафрейм['столбец'] > 30000) & (датафрейм['столбец'] <= 90000)]
# data_ti_medium_percent = (data_ti_medium.count() / 19351)
# data_ti_rich = датафрейм['столбец'].loc[(датафрейм['столбец'] > 90000) & (датафрейм['столбец'] <= 150000)]
# data_ti_rich_percent = (data_ti_rich.count() / 19351)
# data_ti_moneybags = датафрейм['столбец'].loc[(датафрейм['столбец'] > 150000)]
# data_ti_moneybags_percent = (data_ti_moneybags.count() / 19351)
# data_ti_less_than_zero = датафрейм['столбец'].loc[(датафрейм['столбец'] < 0)]
# data_ti_less_than_zero_percent = (data_ti_less_than_zero.count() / 19351)
# data_ti_zero = датафрейм['столбец'].loc[(датафрейм['столбец'] == 0)]
# data_ti_zero_percent = (data_ti_zero.count() / 19351)
# print("Бедные заемщики, составляют: {:.1%}".format(data_ti_poor_percent), "выборки")
# print("Средние заемщики, составляют: {:.1%}".format(data_ti_medium_percent), "выборки")
# print("Богатые заемщики, составляют: {:.1%}".format(data_ti_rich_percent), "выборки")
# print("Сверхбогатые заемщики, составляют: {:.1%}".format(data_ti_moneybags_percent), "выборки")
# print('------------------------------------------')
# print('Количество заемщиков с отрицательным доходом:', data_ti_less_than_zero_percent) # ну хоть так
# print('Количество заемщиков с доходом равным нулю:', data_ti_zero_percent) # радует

Выборка не просто скошено вправо, а сильно скошена вправо.  
Такое ощущение, что тут все жители Тверской и Якиманки.  
Сделаю вот, что - подсчитаю средние значения дохода для "средних", "богатых" и "сверхбогатых", "бедных" исключу - статистически не значимы.  
Беру средние, так как, слишком много выбросов, непонятно, где именно отсутствуют значения в "богатых" или в "бедных", медиана покажет слишком жесткие значения, детализировать по диапазону далее, трудозатратно. Впрочем, на медианы, тоже посмотрю, должны быть близки со средними.   

In [19]:
# data_ti_medium_mean = (датафрейм['столбец'].loc[(датафрейм['столбец'] > 30000) & (датафрейм['столбец'] <= 90000)].sum()) / (data_ti_medium.count())
# data_ti_rich_mean = (датафрейм['столбец'].loc[(датафрейм['столбец'] > 90000) & (датафрейм['столбец'] <= 150000)].sum()) / (data_ti_rich.count())
# data_ti_moneybags_mean = (датафрейм['столбец'].loc[(датафрейм['столбец'] > 150000)].sum()) / (data_ti_moneybags.count())
# print(data_ti_medium_mean)
# print(data_ti_rich_mean)
# print(data_ti_moneybags_mean)
# print('---------------------------')
# data_ti_medium_median = data_ti_medium.median()
# data_ti_rich_median = data_ti_rich.median()
# data_ti_moneybags_median = data_ti_moneybags.median()
# print(data_ti_medium_median)
# print(data_ti_rich_median)
# print(data_ti_moneybags_median)
# print('---------------------------')
# print("Значения средних и медины близки, это очень хорошо - истина где - то рядом")

Что-то нарисовывается. Высчитаю средне - взвешеную величину, в зависимости от доли вклада дохода каждого типа заемщика в выборку.  
И заменю этой величиной пропущенные значения.  
Проверю датафрейм на пропуски.  

In [20]:
# data_ti_average_income = ((data_ti_medium_mean * 17.2))/100 + ((data_ti_rich_mean * 35.2))/100 + ((data_ti_moneybags_mean * 47.5))/100
# print(data_ti_average_income)
# датафрейм['столбец'] = датафрейм['столбец'].fillna(value = data_ti_average_income)
# print(data.info())

Заменю тип данных в столбце "столбец" и еще раз посмотрю на датафрейм

In [21]:
# датафрейм['столбец'] = датафрейм['столбец'].astype('int')
# print(data.info())

Ну вот теперь все числовые значения приведены к целочисленным, с типами "object" буду работать далее

Краткие выводы:  
 - Столбец "столбец" - заменил отрицательные значения на положительные,  
 - Столбец "столбец" - взял отрицальные значения по модулю, выяснил кто из заемщиков искажал общую картину, заменил пропущенные значения на средние, исходя из возраста,  
 - Столбец "столбец" заменил пропущеные значения на средние,  
 - Столбцы "столбец" и "столбец" удалил,  
 - Столбец "столбец" заполнил пропуски, привел к целочисленным значениям  

### Обработка дубликатов

Большое количество дубликатов обнаружилось в столбцах с категориальными переменными - "столбец", "столбец", "столбец", "столбец".  
Удалил дубликаты методом drop_duplicates  

In [22]:
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)

Приведу столбцы "столбец", "столбец", "столбец" и "столбец" к нижнему регистру и снова удалю дубликаты

In [23]:
# датафрейм['столбец'] = датафрейм['столбец'].str.lower()
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)
# датафрейм['столбец'] = датафрейм['столбец'].str.lower()
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)
# датафрейм['столбец'] = датафрейм['столбец'].str.lower()
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)
# датафрейм['столбец'] = датафрейм['столбец'].str.lower()
# датафрейм['столбец'].drop_duplicates().reset_index(drop = True)

Привел данные к нижнему регистру.
Причин появлений дубликатов может быть несколько:
- при заполнении электронной анекты, заемщики не выбирают прописанные значения, а заполянют данные в самостоятельно,
- аналогичная ситуация, когда сотрудник банка переносит данные с бумажной версии в ПО,
- данные получены из разных источников, из разного ПО,
- данные обрабатывались ранее с другими целями

### Лемматизация

Как видно из таблицы, разброс вариантов заполнения, даже семейного положения очень велик, не говоря уж о цели кредита, проведу лемматизацию - приведу словоформы к лемме — её нормальной (словарной) форме.  
Лемматизацию проведу простым методом, применю метод unique()  

Сначала для "столбец", посмотрю все возможные значения и проверю на уникальность  

In [24]:
# data_family_status_unique = датафрейм['столбец'].unique()
# print(data_family_status_unique)

Получилось, всего 5 вариантов - 'женат / замужем' - понятно, есть брак, 'гражданский брак' - это не брак, а сожительство, будем считать, что юридических последствий не имеет, а значит это не "брак", с остальными и так понятно - это не "брак"

Проведу аналогичную операцию для столбца 'столбец'  

In [25]:
# data_purpose_unique = датафрейм['столбец'].unique()
# print(data_purpose_unique)

В столбце 'столбец' все, намного интереснее, вариантов получения кредита, значительно больше.  
Заменю данные и проверю еще раз.  
Для обоих столбцов буду использовать метод replace  

In [26]:
# датафрейм['столбец'] = датафрейм['столбец'].replace(['женат / замужем'], 'брак')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['гражданский брак', 'вдовец / вдова', 'в разводе', 'не женат / не замужем'], 'не_брак')

Еще раз проверю данные методом unique()

In [27]:
# data_family_status_unique = датафрейм['столбец'].unique()
# print(data_family_status_unique)

Получилось, аналогично для столбца 'столбец'  

In [28]:
# датафрейм['столбец'] = датафрейм['столбец'].replace(['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля'], 'автомобиль')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['покупка коммерческой недвижимости', 'операции с коммерческой недвижимостью', 'покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'строительство жилой недвижимости', 'операции со своей недвижимостью', 'покупка своего жилья', 'операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилью', 'жилье'], 'недвижимость')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['дополнительное образование', 'заняться образованием', 'получение образования', 'получение дополнительного образования', 'получение высшего образования', 'профильное образование', 'высшее образование', 'заняться высшим образованием'], 'образование')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['сыграть свадьбу', 'на проведение свадьбы',], 'свадьба')

In [29]:
# data_purpose_unique = датафрейм['столбец'].unique()
# print(data_purpose_unique)

Аналогично для столбца "столбец"

In [30]:
# data_education_unique = датафрейм['столбец'].unique()
# print(data_education_unique)

In [31]:
# датафрейм['столбец'] = датафрейм['столбец'].replace(['высшее', 'ученая степень'], 'высшее')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['среднее', 'неоконченное высшее', 'начальное'], 'не_высшее')
# data_education_unique = датафрейм['столбец'].unique()
# print(data_education_unique)

Аналогично для столбца "столбец"  

In [32]:
# data_income_type_unique = датафрейм['столбец'].unique()
# print(data_income_type_unique)

In [33]:
# датафрейм['столбец']= датафрейм['столбец'].replace(['сотрудник', 'госслужащий'], 'наемный_сотрудник')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['пенсионер'], 'пенсионер')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['компаньон', 'предприниматель'], 'предприниматель')
# датафрейм['столбец'] = датафрейм['столбец'].replace(['безработный', 'студент', 'в декрете'], 'безработный')                                        
# data_income_type_unique = датафрейм['столбец'].unique()
# print(data_income_type_unique)

Получилось! Есть 2 вариант семейного положения - брак и не_брак, 4 варианта получения кредита - недвижимость, автомобиль, образование, свадьба и 2 варианта образования -  высшее и не_высшее, 4 вариант занятости - наемный_сотрудник, пенсионер, предприниматель, безработный

### Вывод

Пропуски удалил, произвел замену, лемматезировал данные в столбцах "столбец", "столбец", "столбец", "столбец".  
Библиотеками для лемматезации не пользовался, так данные были представлены не в разных падежах, например, а даже, если и в разных, то это не важно, важно было их сгруппировать для дальнейшего анализа. # import this  

### Категоризация данных

Приведу данные к значениям, пригодным для анализа.    
Все имеющие детей, присвою значение "1", всем не имеющим, соответственно "0".    
Аналогично по столбцу "столбец" все имевшие просрочку, получают значение "1", всем не имевшим, соответственно "0".    
Сделано, больше, для "очистки совести", тк там и так верные данные.    
Аналогично сделаю по столбцу "столбец".     
Ранее, в столбце "столбец", методом replace провел замену данных с 'приобретение автомобиля', 'на покупку подержанного автомобиля' и тд на "автомобиль", аналогично для значений 'покупка коммерческой недвижимости' и тд на "недвижимость", 'дополнительное образование' и тд на "образование", "'сыграть свадьбу'" на "свадьба".      
В столбце "столбец", "столбец" и "столбец" произвел замену аналогичным образом.    
Посмотрел на таблицу    

In [34]:
# датафрейм['столбец'].loc[(датафрейм['столбец'] != 0)] = 1
# датафрейм['столбец'].loc[(датафрейм['столбец'] == 0)] = 0
# датафрейм['столбец'].loc[(датафрейм['столбец'] != 0)] = 1
# датафрейм['столбец'].loc[(датафрейм['столбец'] == 0)] = 0
# датафрейм['столбец'].loc[(датафрейм['столбец'] == 'F')] = 'F'
# датафрейм['столбец'].loc[(датафрейм['столбец'] == 'M')] = 'M'
# print(data.head())

Посмотрел на таблицу, все получилось  

Создам словари на основе, ранее категоризированных данных.  
Будет интересовать платеженая дисциплина заемщика в зависимости от наличия детей, уровня дохода, целей получения кредита  

Для заемщиков, имеющих или не имеющих детей

In [35]:
# data_children_grouped = датафрейм.groupby('столбец').agg({'debt': ['count','sum']})
# print(data_children_grouped)

Для заемщиков в зависимости отт уровня дохода, где  
1 - бедные, с доходом до 30000 руб., включительно,  
2 - средние - с доходом от 30000 до 90000 руб., включительно,  
3 - богатые - с доходом от 90000 до 150000 руб., включительно,  
4 - сверхбогатые - с доходом от 150000 руб.  

In [36]:
# датафрейм['столбец'].loc[(датафрейм['столбец'] <= 30000)] = 1
# датафрейм['столбец'].loc[(датафрейм['столбец'] > 30000) & (датафрейм['столбец'] <= 90000)] = 2
# датафрейм['столбец'].loc[(датафрейм['столбец'] > 90000) & (датафрейм['столбец'] <= 150000)] = 3
# датафрейм['столбец'].loc[(датафрейм['столбец'] > 150000)] = 4
# data_ti_grouped = датафрейм.groupby('столбец').agg({'debt': ['count','sum']})
# print(data_ti_grouped)

In [37]:
# data_family_status_grouped = датафрейм.groupby('столбец').agg({'debt': ['count','sum']})
# print(data_family_status_grouped)

Для заемщиков в зависимости от цели кредитования

In [38]:
# data_purpose_grouped = датафрейм.groupby('столбец').agg({'debt': ['count','sum']})
# print(data_purpose_grouped)

Уровни зарплат выбирал на основе данных Госкомстата, отсюда: https://www.bnkomi.ru/data/news/59037/, но сгруппировал по более крупным категориям.

Заемщиков по уровню зарплат разделил, в качестве основы, использовав официальные данные, а не данные датафрейма, тк выборка нерелевантна реальному положению дел в РФ - сильно скошена вправо.  
А затем уже категоризировал заемщиков из датафрейм в соответствии с утвержденными стандартами.  
Насколько я понимаю задание - надо на основании ограниченной выборки построить скоринговую модель, то есть, в дальнейшем наложить выводы из этих данных на новых заемщиков, а не проанализировать старых.  

Выводы после категоризации и создания словарей - уже невооруженным взглядом видно, кто из заемщиков более дисциплинирован.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построю сводную таблицу для заемщиков, имеющих и не имеющих детей по уровню дохода

In [39]:
# data_pivot_children = датафрейм.pivot_table(index=['столбец'], columns='столбец', values='debt', aggfunc= lambda x: x.sum() / data['debt'].count())
# data_pivot_children.rename(columns={0: 'n_children', 1: 'y_children'}, inplace=True)
# data_pivot_children['n_children'] = data_pivot_children['n_children'] * 100
# data_pivot_children['y_children'] = data_pivot_children['y_children'] * 100
# print(data_pivot_children)

### Вывод

Есть, заемщики всех категорий доходов, имеющме детей, допускали просрочку в заметно меньшем количестве случаев

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построю сводную таблицу для заемщиков, находящихся и не находящихся в браке, по уровню дохода

In [40]:
# data_pivot_family_status = датафрейм.pivot_table(index=['столбец'], columns='столбец', values='debt', aggfunc= lambda x: x.sum() / data['debt'].count())
# data_pivot_family_status['брак'] = data_pivot_family_status['брак'] * 100
# data_pivot_family_status['не_брак'] = data_pivot_family_status['не_брак'] * 100
# print(data_pivot_family_status)

### Вывод

Заемщики, находящиеся в браке, несколько хуже справляются со своими обязательствами по возврату кредита, независимо от уровня дохода

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Построю сводную таблицу для заемщиков, по уровню дохода, с учетом семейного статуса

In [41]:
# data_pivot_total_income = датафрейм.pivot_table(index=['столбец], columns='столбец', values='debt', aggfunc= lambda x: x.sum() / data['debt'].count())
# data_pivot_total_income.rename(columns={1: 'poor', 2: 'medium', 3: 'rich', 4: 'moneybags'}, inplace=True)
# data_pivot_total_income['poor'] = data_pivot_total_income['poor'] * 100
# data_pivot_total_income['medium'] = data_pivot_total_income['medium'] * 100
# data_pivot_total_income['rich'] = data_pivot_total_income['rich'] * 100
# data_pivot_total_income['moneybags'] = data_pivot_total_income['moneybags'] * 100
# print(data_pivot_total_income)

### Вывод

Самыми недисциплинированными заемщиками оказались сверхбогатые заемщики с доходом выше 150 тыс. руб., находящиеся в браке.

- Как разные цели кредита влияют на его возврат в срок?

Построю сводную таблицу для заемщиков, по целям получения кредита, с учетом дохода

In [42]:
# data_pivot_purpose = датафрейм.pivot_table(index=['столбец'], columns='столбец', values='столбец', aggfunc= lambda x: x.sum() / data['debt'].count())
# data_pivot_purpose['автомобиль'] = data_pivot_purpose['автомобиль'] * 100
# data_pivot_purpose['недвижимость'] = data_pivot_purpose['недвижимость'] * 100
# data_pivot_purpose['образование'] = data_pivot_purpose['образование'] * 100
# data_pivot_purpose['свадьба'] = data_pivot_purpose['свадьба'] * 100
# print(data_pivot_purpose)

### Вывод

Самыми недисциплинрованными оказались сверхбогатые заемщики, берущие кредиты на недвижимость, очевидно скрытое кредитование бизнеса, самыми дисциплинированными оказались средние бедные и средние заемщики, вне зависимости от целей кредитования.  

Сведу все данные в одну сводную таблицу и оценю дисицплиниованность заемщиков по совокупности факторов.

In [43]:
# data_pivot = датафрейм.pivot_table(index=['столбец', 'столбец', 'столбец'], columns='столбец', values='debt', \
# aggfunc = (lambda x: x.sum() / датафрейм['столбец'].count(), margins=True)
# data_pivot.rename(columns={0: 'n_children', 1: 'y_children'}, inplace=True)
# data_pivot['n_children'] = data_pivot['n_children'] * 100
# data_pivot['y_children'] = data_pivot['y_children'] * 100
# data_pivot.sort_values(by=['n_children','y_children'], ascending=False)
# print(data_pivot) # см. http://datareview.info/article/svodnyie-tablitsyi-v-python/

### Шаг 4. Общий вывод

В работу попала таблица, содержащая данные о платежной дисциплине заемщиков, содержащая, также даннее о количестве детей, трудовом стаже, возрасте  и образовании клиента, семейном положении, поле, тпиу занятости, доходу и целям получения кредита, помимо этого данные содержали id - индентификтоары этих данных.    

Поскольку в задании стоял вопрос о нахождении зависимости платежной дисциплины заемщика от наличия детей, уровня дохода, семейного статуса и целей получения кредита, то лишние столбцы были удалены.    

В столбце с уровнем дохода обнаружилось большое количество пропусков.  
Поскольку данные были смешены в сторону заемщиков с выскоим уровнем дохода, то просто взять среднее или медиану от всего массива было бы некорректно. Пришлось разбить заемщиков на подкатегории - бедные, средние, богаты, сверхбогаты и уже найти средний уровень дохода в каждой подкатегории.  
Выяснилось, что в данной выборке доля бедных заемщиков очень низка, всего 0.1% выборки. Очевидно, что это данные по заемщикам из крупного гоорода - Москва, Спб, но, точно не вся РФ. Поэтому экстраполировать выводы о благонадежности заемщиков на всю клиентскую базу было бы неверно, надо рассчитывать, отдельно для каждого региона, так как наблюдается очень сильная дифференцация по уровню дохода.    

Можно, использовать и эти выводы, но с учетом коэффициентов по уровню жизни в том или ином регионе. Пропущенные значения были заменены на средние, но с учетом веса кажой категории заемщиков в выборке. Посмотрел, также на медианные значения, они оказались близки со средними, значит дифференцация была сделана "близко к тексту".    

В данных о количестве детей и семейном положении, данные были заменены на '0' и '1' и "брак" и "не_брак", соответственно, чтобы облегчить и упростить таблицу. В данных о целях кредитования, не встретились цели - "полет на Луну", "мир во всем мире", что облегчило их категоризация на всего 4 цели - "недвижимость' 'автомобиль' 'образование' 'свадьба".    

Были категоризированы и другие столбцы.    

Вся информация представлена в сводной таблице, в которой левый столбец, это соответственно, заемщики в зависмости от уровня дохода, так как это самый первичный показатель, влиящий на возможность заемщика исполнять свтои обязательства, следующий столбец - цели получения кредита, следующий - семейное положение заемщиков и 2 столбца с указанием процента заемщиков не имеющими детей и процентов заемщиков, имеющих детей и допустивших просрочку, от общего количества заемщиков.    

Как видно, из таблицы, самыми дисциплинированными заемщиками это заемщики с низким и средним уровнем дохода (от 0 до 90000 рублей), состоящие в браке, имеющие детей и берущие кредиты на автомобили и недвижимость.   
Также, весьма дисциплинированными заемщиками являются не имеющие детей, находящиеся как в браке, так и не в браке, берущие кредиты на образование, свадьбу. "Синие воротнички", офисный планктон - им есть что терять. Отдельно стоит выделит заемщиков 1 и 2 категории, берущих кредиты на образование, тоже довольно низкий уровень просрочки.    

Самыми недисциплинированными заемщиками, являтся очень обеспеченные заемщики, находящиеся как в браке, так и не в браке, в основном не имеющие детей и берущие кредиты на недвижимость и автомобили - скорее всего это скрытое кредитование бизнеса.    

В целом, данные позволяют посмотреть на заемщиков под разными углами, в зависимости от разных показателей, то есть сделать более узкие срезы. Можно, также, разделить заемщиков по количеству детей, по уровню образованию, по занятости и также провести анализ.    